In [1]:
import torch.nn as nn
import torchvision
import torch.utils.data
from models import BasicParkingNet

In [2]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(),
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        torchvision.transforms.Resize((40, 60)),
        torchvision.transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [3]:
# Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BasicParkingNet().to(device).train()
criterion = nn.BCEWithLogitsLoss()
num_episodes = 100

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        model.optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        model.optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/basic_parking_net.pth")
        print("\tSaving!")


Episode 0, loss: 35.62409131228924
	Saving!
Episode 1, loss: 14.790595028549433
	Saving!
Episode 2, loss: 11.380870055407286
	Saving!
Episode 3, loss: 7.586602408438921
	Saving!
Episode 4, loss: 6.198978269472718
	Saving!
Episode 5, loss: 4.078187359496951
	Saving!
Episode 6, loss: 4.686412330716848
Episode 7, loss: 4.650973599404097
Episode 8, loss: 4.2324929824098945
Episode 9, loss: 3.8588416380807757
	Saving!
Episode 10, loss: 3.5761344763450325
	Saving!
Episode 11, loss: 2.6543336980976164
	Saving!
Episode 12, loss: 4.236465222667903
Episode 13, loss: 2.3575597130693495
	Saving!
Episode 14, loss: 3.5081052859313786
Episode 15, loss: 2.789002433884889
Episode 16, loss: 4.402275493368506
Episode 17, loss: 4.20146489655599
Episode 18, loss: 3.624187112785876
Episode 19, loss: 6.1010601576417685
Episode 20, loss: 4.476664560381323
Episode 21, loss: 3.523781495168805
Episode 22, loss: 3.725815744139254
Episode 23, loss: 3.5749977342784405
Episode 24, loss: 2.3880474970210344
Episode 25